In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm, spearmanr, kendalltau
from scipy.interpolate import griddata 
from scipy import interpolate
import sys

import lmom as lmom
from datetime import datetime

Selection of the marginal distribution according to goodness-of-fit measures and the log plots for each of the annual maxima sets (using the full data sets)

In [ ]:
c = 68
#Without A&V
# op = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HIDnew\05_HYDzoom" + r"\C%d_" %c + r"disc1d_"
#With A&V
op = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ2" + r"\C%d_" %c + r"disc1d_"

AM1 = pd.read_pickle(op + "AM1.pkl")
AM2 = pd.read_pickle(op + "AM2.pkl")
AM3 = pd.read_pickle(op + "AM3.pkl")

In [ ]:
#Empirical distribution function
def ecdf(data):
    """ Compute Empirical CDF """
    x = np.sort(data)
    n = x.size
    rank = np.arange(1, n+1)
    y = rank / (n+1)
    return x, y

#root mean squared error.
def RMSE(yfit, y):
    E = np.sqrt(1 / len(y) * (np.sum((y - yfit) ** 2)))
    return E

#AIC function
def AIC(yfit, y, k):
    n = len(y)
    S = np.sum((y - yfit) ** 2)
    aic = n * np.log(S / n) + 2 * k
    return aic

#BIC function
def BIC(yfit, y, k):
    n = len(y)
    S = np.sum((y - yfit) ** 2)
    bic = n * np.log(S / n) + k * np.log(n)
    return bic

Set 1

In [ ]:
#Marginal distributions - Parameter estimation
                                        #Method of fitting 'MM' Method of moments 'MLE' Maximum Likelihood Estimation

LMU1 = lmom.samlmu(AM1['Q_S1'])
LMU2 = lmom.samlmu(AM1['Q_S2'])
#Gumbel
gum_fit1 = lmom.pelgum(LMU1)
gum_fit2 = lmom.pelgum(LMU2)
# GEV
gev_fit1 = lmom.pelgev(LMU1)
gev_fit2 = lmom.pelgev(LMU2)
#General pareto
gpa_fit1 = lmom.pelgpa(LMU1)
gpa_fit2 = lmom.pelgpa(LMU2)
#Normal
nor_fit1 = lmom.pelnor(LMU1)
nor_fit2 = lmom.pelnor(LMU2)
#Empirical
xed1, yed1 = ecdf(AM1['Q_S1'])
xed2, yed2 = ecdf(AM1['Q_S2'])

#CDFs
gum_cdf1 = np.array([])
gum_cdf2 = np.array([])
gev_cdf1 = np.array([])
gev_cdf2 = np.array([])
gpa_cdf1 = np.array([])
gpa_cdf2 = np.array([])
nor_cdf1 = np.array([])
nor_cdf2 = np.array([])

# x1 = np.linspace(AM1['Q_S1'].min(), AM1['Q_S1'].max(), 100)
# x2 = np.linspace(AM1['Q_S2'].min(), AM1['Q_S2'].max(), 100)

for i in range(len(xed1)):
    #Gumbel
    gum_cdf1 = np.hstack((gum_cdf1, lmom.cdfgum(xed1[i], gum_fit1)))
    gum_cdf2 = np.hstack((gum_cdf2, lmom.cdfgum(xed2[i], gum_fit2)))
    #GEV
    gev_cdf1 = np.hstack((gev_cdf1, lmom.cdfgev(xed1[i], gev_fit1)))
    gev_cdf2 = np.hstack((gev_cdf2, lmom.cdfgev(xed2[i], gev_fit2)))
    #General pareto
    gpa_cdf1 = np.hstack((gpa_cdf1, lmom.cdfgpa(xed1[i], gpa_fit1)))
    gpa_cdf2 = np.hstack((gpa_cdf2, lmom.cdfgpa(xed2[i], gpa_fit2)))
    #Normal 
    nor_cdf1 = np.hstack((nor_cdf1, lmom.cdfnor(xed1[i], nor_fit1)))
    nor_cdf2 = np.hstack((nor_cdf2, lmom.cdfnor(xed2[i], nor_fit2)))


cdfs_ = gum_cdf1, gum_cdf2, gev_cdf1, gev_cdf2, gpa_cdf1, gpa_cdf2, nor_cdf1, nor_cdf2
cdfs_text = ['gum_cdf1', 'gum_cdf2', 'gev_cdf1', 'gev_cdf2', 'gpa_cdf1', 'gpa_cdf2', 'nor_cdf1', 'nor_cdf2']
RMSE_ = RMSE(gum_cdf1, yed1), RMSE(gum_cdf2, yed2), RMSE(gev_cdf1, yed1), RMSE(gev_cdf2, yed2), RMSE(([0 if d is None else d for d in gpa_cdf1]), yed1), RMSE(([0 if d is None else d for d in gpa_cdf2]), yed2), RMSE(nor_cdf1, yed1), RMSE(nor_cdf2, yed2)
AIC_ = AIC(gum_cdf1, yed1, len(gum_fit1)), AIC(gum_cdf2, yed2, len(gum_fit2)), AIC(gev_cdf1, yed1, len(gev_fit1)), AIC(gev_cdf2, yed2, len(gev_fit2)), AIC(([0 if d is None else d for d in gpa_cdf1]), yed1, len(gpa_fit1)), AIC(([0 if d is None else d for d in gpa_cdf2]), yed2, len(gpa_fit2)), AIC(nor_cdf1, yed1, len(nor_fit1)), AIC(nor_cdf2, yed2, len(nor_fit2))
BIC_ = BIC(gum_cdf1, yed1, len(gum_fit1)), BIC(gum_cdf2, yed2, len(gum_fit2)), BIC(gev_cdf1, yed1, len(gev_fit1)), BIC(gev_cdf2, yed2, len(gev_fit2)), BIC(([0 if d is None else d for d in gpa_cdf1]), yed1, len(gpa_fit1)), BIC(([0 if d is None else d for d in gpa_cdf2]), yed2, len(gpa_fit2)), BIC(nor_cdf1, yed1, len(nor_fit1)), BIC(nor_cdf2, yed2, len(nor_fit2))

Title = 'Gumbel (GUM)', 'Generalised Extreme Value (GEV)', 'Generalised Pareto (GPA)', 'Normal (NOR)'
plt.figure(figsize=(12, 12))
plt.suptitle('Set 1 | S1max-S2conc', fontsize=22, y=1.02) #'Set 2 | S2max-S1conc' 'Set 3 | Cmax (C=S1+S2)'
# plt.title('Set 1 | S1max-S2conc', fontsize=22) 
#for i in dis_:
count1 = -1
count2 = 0
for j in range(4):
    count1 = count1 + 2
    plt.subplot(4,2, count1)
    plt.plot(xed1, yed1, 'm+', label='Empirical distribution 1')
    plt.plot(xed1, cdfs_[count1-1], 'b', label=f'{cdfs_text[count1-1]} \nRMSE={RMSE_[count1-1]:.4f} \nAIC={AIC_[count1-1]:.0f} \nBIC={BIC_[count1-1]:.0f}')
    plt.title(f'{Title[j]} | Rhine river', fontsize=18)
    plt.xlabel('Discharge (m3/s)', fontsize=14)
    plt.ylabel('Probability', fontsize=14) #Q m3/s
    plt.legend(fontsize=14, loc=4)
    plt.grid()
    count2 = count2 + 2
    plt.subplot(4,2, count2)
    plt.plot(xed2, yed2, 'm+', label='Empirical distribution 2')
    plt.plot(xed2, cdfs_[count2-1], 'b', label=f'{cdfs_text[count2-1]} \nRMSE={RMSE_[count2-1]:.4f} \nAIC={AIC_[count2-1]:.0f} \nBIC={BIC_[count2-1]:.0f}')
    plt.title(f'{Title[j]} | Main river', fontsize=18)
    plt.xlabel('Discharge (m3/s)', fontsize=14)
    plt.ylabel('Probability', fontsize=14) #Q m3/s
    plt.legend(fontsize=14, loc=4)
    plt.grid()
plt.tight_layout()

In [ ]:
#CDFs
gum_cdf1 = np.array([])
gum_cdf2 = np.array([])
gev_cdf1 = np.array([])
gev_cdf2 = np.array([])
gpa_cdf1 = np.array([])
gpa_cdf2 = np.array([])
nor_cdf1 = np.array([])
nor_cdf2 = np.array([])

x1 = np.linspace(AM1['Q_S1'].min(), AM1['Q_S1'].max()+1000, 100)
x2 = np.linspace(AM1['Q_S2'].min(), AM1['Q_S2'].max()+1000, 100)

for i in range(100):
    #Gumbel
    gum_cdf1 = np.hstack((gum_cdf1, lmom.cdfgum(x1[i], gum_fit1)))
    gum_cdf2 = np.hstack((gum_cdf2, lmom.cdfgum(x2[i], gum_fit2)))
    #GEV
    gev_cdf1 = np.hstack((gev_cdf1, lmom.cdfgev(x1[i], gev_fit1)))
    gev_cdf2 = np.hstack((gev_cdf2, lmom.cdfgev(x2[i], gev_fit2)))
    #General pareto
    gpa_cdf1 = np.hstack((gpa_cdf1, lmom.cdfgpa(x1[i], gpa_fit1)))
    gpa_cdf2 = np.hstack((gpa_cdf2, lmom.cdfgpa(x2[i], gpa_fit2)))
    #Normal
    nor_cdf1 = np.hstack((nor_cdf1, lmom.cdfnor(x1[i], nor_fit1)))
    nor_cdf2 = np.hstack((nor_cdf2, lmom.cdfnor(x2[i], nor_fit2)))

gpa_cdf1_ = np.zeros(len(gpa_cdf1))
gpa_cdf2_ = np.zeros(len(gpa_cdf2))

for i in range(len(gpa_cdf1_)):
    if gpa_cdf1[i] == None:
        gpa_cdf1_[i] = 0.00001
    else:
        gpa_cdf1_[i] = gpa_cdf1[i]
    if gpa_cdf2[i] == None:
        gpa_cdf2_[i] = 0.00001
    else:
        gpa_cdf2_[i] = gpa_cdf2[i]
# gpa_cdf1_ = ([0 if d is None else d for d in gpa_cdf1])
# gpa_cdf2_ = ([0 if d is None else d for d in gpa_cdf2])

#print(gpa_cdf2)
cdfs_ = gum_cdf1, gum_cdf2, gev_cdf1, gev_cdf2, gpa_cdf1_, gpa_cdf2_, nor_cdf1, nor_cdf2
cdfs_text = ['gum_cdf1', 'gum_cdf2', 'gev_cdf1', 'gev_cdf2', 'gpa_cdf1', 'gpa_cdf2', 'nor_cdf1', 'nor_cdf2']


Title = 'Gumbel (GUM)', 'Generalised Extreme Value (GEV)', 'Generalised Pareto (GPA)', 'Normal (NOR)'
plt.figure(figsize=(12, 12))
plt.suptitle('Set 1 | S1max-S2conc', fontsize=22, y=1.02) #'Set 2 | S2max-S1conc' 'Set 3 | Cmax (C=S1+S2)'
#for i in dis_:
count1 = -1
count2 = 0
for j in range(4):
    count1 = count1 + 2
    plt.subplot(4,2, count1)
    plt.plot(1/(1-cdfs_[count1-1]), x1, 'b', label=f'{cdfs_text[count1-1]} \nRMSE={RMSE_[count1-1]:.4f} \nAIC={AIC_[count1-1]:.0f} \nBIC={BIC_[count1-1]:.0f}')
    plt.plot(1/(1-yed1), xed1, 'm+', label='Empirical distribution 1')
    plt.title(f'{Title[j]} | Rhine river', fontsize=18)
    plt.xlabel('Return period (years)')
    plt.ylabel('Discharge (m3/s)')
    plt.xscale('log')
    plt.xlim(right=10**6)
    plt.legend()
    plt.grid()
    count2 = count2 + 2
    plt.subplot(4,2, count2)
    plt.plot(1/(1-cdfs_[count2-1][4:]), x2[4:], 'b', label=f'{cdfs_text[count2-1]} \nRMSE={RMSE_[count2-1]:.4f} \nAIC={AIC_[count2-1]:.0f} \nBIC={BIC_[count2-1]:.0f}')
    plt.plot(1/(1-yed2), xed2, 'm+', label='Empirical distribution 2')
    plt.title(f'{Title[j]} | Main river', fontsize=18)
    plt.xlabel('Return period (years)')
    plt.ylabel('Discharge (m3/s)')
    plt.xscale('log')
    plt.xlim(right=10**6)
    plt.legend()
    plt.grid()
plt.tight_layout()

In [ ]:
print('RMSE-Q_S1')
print(RMSE(gum_cdf1, yed1), RMSE(gev_cdf1, yed1), RMSE(([0 if d is None else d for d in gpa_cdf1]), yed1))
print('RMSE-Q_S2')
print(RMSE(gum_cdf2, yed2), RMSE(gev_cdf2, yed2), RMSE(([0 if d is None else d for d in gpa_cdf2]), yed2))
print('AIC-Q_S1')
print(AIC(gum_cdf1, yed1, len(gum_fit1)), AIC(gev_cdf1, yed1, len(gev_fit1)), AIC(([0 if d is None else d for d in gpa_cdf1]), yed1, len(gpa_fit1)))
print('AIC-Q_S2')
print(AIC(gum_cdf2, yed2, len(gum_fit2)), AIC(gev_cdf2, yed2, len(gev_fit2)), AIC(([0 if d is None else d for d in gpa_cdf2]), yed2, len(gpa_fit2)))
print('BIC-Q_S1')
print(BIC(gum_cdf1, yed1, len(gum_fit1)), BIC(gev_cdf1, yed1, len(gev_fit1)), BIC(([0 if d is None else d for d in gpa_cdf1]), yed1, len(gpa_fit1)))
print('BIC-Q_S2')
print(BIC(gum_cdf2, yed2, len(gum_fit2)), BIC(gev_cdf2, yed2, len(gev_fit2)), BIC(([0 if d is None else d for d in gpa_cdf2]), yed2, len(gpa_fit2)))
    

Set 2

In [ ]:
#Marginal distributions - Parameter estimation
                                        #Method of fitting 'MM' Method of moments 'MLE' Maximum Likelihood Estimation

LMU1 = lmom.samlmu(AM2['Q_S1'])
LMU2 = lmom.samlmu(AM2['Q_S2'])
#Gumbel
gum_fit1 = lmom.pelgum(LMU1)
gum_fit2 = lmom.pelgum(LMU2)
# GEV
gev_fit1 = lmom.pelgev(LMU1)
gev_fit2 = lmom.pelgev(LMU2)
#General pareto
gpa_fit1 = lmom.pelgpa(LMU1)
gpa_fit2 = lmom.pelgpa(LMU2)
#Normal
nor_fit1 = lmom.pelnor(LMU1)
nor_fit2 = lmom.pelnor(LMU2)
#Empirical
xed1, yed1 = ecdf(AM2['Q_S1'])
xed2, yed2 = ecdf(AM2['Q_S2'])

#CDFs
gum_cdf1 = np.array([])
gum_cdf2 = np.array([])
gev_cdf1 = np.array([])
gev_cdf2 = np.array([])
gpa_cdf1 = np.array([])
gpa_cdf2 = np.array([])
nor_cdf1 = np.array([])
nor_cdf2 = np.array([])

# x1 = np.linspace(AM2['Q_S1'].min(), AM2['Q_S1'].max(), 100)
# x2 = np.linspace(AM2['Q_S2'].min(), AM2['Q_S2'].max(), 100)

for i in range(len(xed1)):
    #Gumbel
    gum_cdf1 = np.hstack((gum_cdf1, lmom.cdfgum(xed1[i], gum_fit1)))
    gum_cdf2 = np.hstack((gum_cdf2, lmom.cdfgum(xed2[i], gum_fit2)))
    #GEV
    gev_cdf1 = np.hstack((gev_cdf1, lmom.cdfgev(xed1[i], gev_fit1)))
    gev_cdf2 = np.hstack((gev_cdf2, lmom.cdfgev(xed2[i], gev_fit2)))
    #General pareto
    gpa_cdf1 = np.hstack((gpa_cdf1, lmom.cdfgpa(xed1[i], gpa_fit1)))
    gpa_cdf2 = np.hstack((gpa_cdf2, lmom.cdfgpa(xed2[i], gpa_fit2)))
   #Normal 
    nor_cdf1 = np.hstack((nor_cdf1, lmom.cdfnor(xed1[i], nor_fit1)))
    nor_cdf2 = np.hstack((nor_cdf2, lmom.cdfnor(xed2[i], nor_fit2)))


cdfs_ = gum_cdf1, gum_cdf2, gev_cdf1, gev_cdf2, gpa_cdf1, gpa_cdf2, nor_cdf1, nor_cdf2
cdfs_text = ['gum_cdf1', 'gum_cdf2', 'gev_cdf1', 'gev_cdf2', 'gpa_cdf1', 'gpa_cdf2', 'nor_cdf1', 'nor_cdf2']
RMSE_ = RMSE(gum_cdf1, yed1), RMSE(gum_cdf2, yed2), RMSE(gev_cdf1, yed1), RMSE(gev_cdf2, yed2), RMSE(([0 if d is None else d for d in gpa_cdf1]), yed1), RMSE(([0 if d is None else d for d in gpa_cdf2]), yed2), RMSE(nor_cdf1, yed1), RMSE(nor_cdf2, yed2)
AIC_ = AIC(gum_cdf1, yed1, len(gum_fit1)), AIC(gum_cdf2, yed2, len(gum_fit2)), AIC(gev_cdf1, yed1, len(gev_fit1)), AIC(gev_cdf2, yed2, len(gev_fit2)), AIC(([0 if d is None else d for d in gpa_cdf1]), yed1, len(gpa_fit1)), AIC(([0 if d is None else d for d in gpa_cdf2]), yed2, len(gpa_fit2)), AIC(nor_cdf1, yed1, len(nor_fit1)), AIC(nor_cdf2, yed2, len(nor_fit2))
BIC_ = BIC(gum_cdf1, yed1, len(gum_fit1)), BIC(gum_cdf2, yed2, len(gum_fit2)), BIC(gev_cdf1, yed1, len(gev_fit1)), BIC(gev_cdf2, yed2, len(gev_fit2)), BIC(([0 if d is None else d for d in gpa_cdf1]), yed1, len(gpa_fit1)), BIC(([0 if d is None else d for d in gpa_cdf2]), yed2, len(gpa_fit2)), BIC(nor_cdf1, yed1, len(nor_fit1)), BIC(nor_cdf2, yed2, len(nor_fit2))

Title = 'Gumbel (GUM)', 'Generalised Extreme Value (GEV)', 'Generalised Pareto (GPA)', 'Normal (NOR)'
plt.figure(figsize=(12, 12))
plt.suptitle('Set 2 | S2max-S1conc', fontsize=22, y=1.02) #'Set 2 | S2max-S1conc' 'Set 3 | Cmax (C=S1+S2)'
# plt.title('Set 1 | S1max-S2conc', fontsize=22) 
#for i in dis_:
count1 = -1
count2 = 0
for j in range(4):
    count1 = count1 + 2
    plt.subplot(4,2, count1)
    plt.plot(xed1, yed1, 'm+', label='Empirical distribution 1')
    plt.plot(xed1, cdfs_[count1-1], 'b', label=f'{cdfs_text[count1-1]} \nRMSE={RMSE_[count1-1]:.4f} \nAIC={AIC_[count1-1]:.0f} \nBIC={BIC_[count1-1]:.0f}')
    plt.title(f'{Title[j]} | Rhine river', fontsize=18)
    plt.xlabel('Discharge (m3/s)', fontsize=14)
    plt.ylabel('Probability', fontsize=14) #Q m3/s
    plt.legend(fontsize=14, loc=4)
    plt.grid()
    count2 = count2 + 2
    plt.subplot(4,2, count2)
    plt.plot(xed2, yed2, 'm+', label='Empirical distribution 2')
    plt.plot(xed2, cdfs_[count2-1], 'b', label=f'{cdfs_text[count2-1]} \nRMSE={RMSE_[count2-1]:.4f} \nAIC={AIC_[count2-1]:.0f} \nBIC={BIC_[count2-1]:.0f}')
    plt.title(f'{Title[j]} | Main river', fontsize=18)
    plt.xlabel('Discharge (m3/s)', fontsize=14)
    plt.ylabel('Probability', fontsize=14) #Q m3/s
    plt.legend(fontsize=14, loc=4)
    plt.grid()
plt.tight_layout()

In [ ]:
#CDFs
gum_cdf1 = np.array([])
gum_cdf2 = np.array([])
gev_cdf1 = np.array([])
gev_cdf2 = np.array([])
gpa_cdf1 = np.array([])
gpa_cdf2 = np.array([])
nor_cdf1 = np.array([])
nor_cdf2 = np.array([])

x1 = np.linspace(AM1['Q_S1'].min(), AM1['Q_S1'].max()+1000, 100)
x2 = np.linspace(AM1['Q_S2'].min(), AM1['Q_S2'].max()+1000, 100)

for i in range(100):
    #Gumbel
    gum_cdf1 = np.hstack((gum_cdf1, lmom.cdfgum(x1[i], gum_fit1)))
    gum_cdf2 = np.hstack((gum_cdf2, lmom.cdfgum(x2[i], gum_fit2)))
    #GEV
    gev_cdf1 = np.hstack((gev_cdf1, lmom.cdfgev(x1[i], gev_fit1)))
    gev_cdf2 = np.hstack((gev_cdf2, lmom.cdfgev(x2[i], gev_fit2)))
    #General pareto
    gpa_cdf1 = np.hstack((gpa_cdf1, lmom.cdfgpa(x1[i], gpa_fit1)))
    gpa_cdf2 = np.hstack((gpa_cdf2, lmom.cdfgpa(x2[i], gpa_fit2)))
    #Normal
    nor_cdf1 = np.hstack((nor_cdf1, lmom.cdfnor(x1[i], nor_fit1)))
    nor_cdf2 = np.hstack((nor_cdf2, lmom.cdfnor(x2[i], nor_fit2)))

gpa_cdf1_ = np.zeros(len(gpa_cdf1))
gpa_cdf2_ = np.zeros(len(gpa_cdf2))

for i in range(len(gpa_cdf1_)):
    if gpa_cdf1[i] == None:
        gpa_cdf1_[i] = 0.00001
    else:
        gpa_cdf1_[i] = gpa_cdf1[i]
    if gpa_cdf2[i] == None:
        gpa_cdf2_[i] = 0.00001
    else:
        gpa_cdf2_[i] = gpa_cdf2[i]
# gpa_cdf1_ = ([0 if d is None else d for d in gpa_cdf1])
# gpa_cdf2_ = ([0 if d is None else d for d in gpa_cdf2])

#print(gpa_cdf2)
cdfs_ = gum_cdf1, gum_cdf2, gev_cdf1, gev_cdf2, gpa_cdf1_, gpa_cdf2_, nor_cdf1, nor_cdf2
cdfs_text = ['gum_cdf1', 'gum_cdf2', 'gev_cdf1', 'gev_cdf2', 'gpa_cdf1', 'gpa_cdf2', 'nor_cdf1', 'nor_cdf2']


Title = 'Gumbel (GUM)', 'Generalised Extreme Value (GEV)', 'Generalised Pareto (GPA)', 'Normal (NOR)'
plt.figure(figsize=(12, 12))
plt.suptitle('Set 2 | S2max-S1conc', fontsize=22, y=1.02) #'Set 2 | S2max-S1conc' 'Set 3 | Cmax (C=S1+S2)'
#for i in dis_:
count1 = -1
count2 = 0
for j in range(4):
    count1 = count1 + 2
    plt.subplot(4,2, count1)
    plt.plot(1/(1-cdfs_[count1-1]), x1, 'b', label=f'{cdfs_text[count1-1]} \nRMSE={RMSE_[count1-1]:.4f} \nAIC={AIC_[count1-1]:.0f} \nBIC={BIC_[count1-1]:.0f}')
    plt.plot(1/(1-yed1), xed1, 'm+', label='Empirical distribution 1')
    plt.title(f'{Title[j]} | Rhine river', fontsize=18)
    plt.xlabel('Return period (years)')
    plt.ylabel('Discharge (m3/s)')
    plt.xscale('log')
    plt.xlim(right=10**6)
    plt.legend()
    plt.grid()
    count2 = count2 + 2
    plt.subplot(4,2, count2)
    plt.plot(1/(1-cdfs_[count2-1][4:]), x2[4:], 'b', label=f'{cdfs_text[count2-1]} \nRMSE={RMSE_[count2-1]:.4f} \nAIC={AIC_[count2-1]:.0f} \nBIC={BIC_[count2-1]:.0f}')
    plt.plot(1/(1-yed2), xed2, 'm+', label='Empirical distribution 2')
    plt.title(f'{Title[j]} | Main river', fontsize=18)
    plt.xlabel('Return period (years)')
    plt.ylabel('Discharge (m3/s)')
    plt.xscale('log')
    plt.xlim(right=10**6)
    plt.legend()
    plt.grid()
plt.tight_layout()

In [ ]:
print('RMSE-Q_S1')
print(RMSE(gum_cdf1, yed1), RMSE(gev_cdf1, yed1), RMSE(([0 if d is None else d for d in gpa_cdf1]), yed1))
print('RMSE-Q_S2')
print(RMSE(gum_cdf2, yed2), RMSE(gev_cdf2, yed2), RMSE(([0 if d is None else d for d in gpa_cdf2]), yed2))
print('AIC-Q_S1')
print(AIC(gum_cdf1, yed1, len(gum_fit1)), AIC(gev_cdf1, yed1, len(gev_fit1)), AIC(([0 if d is None else d for d in gpa_cdf1]), yed1, len(gpa_fit1)))
print('AIC-Q_S2')
print(AIC(gum_cdf2, yed2, len(gum_fit2)), AIC(gev_cdf2, yed2, len(gev_fit2)), AIC(([0 if d is None else d for d in gpa_cdf2]), yed2, len(gpa_fit2)))
print('BIC-Q_S1')
print(BIC(gum_cdf1, yed1, len(gum_fit1)), BIC(gev_cdf1, yed1, len(gev_fit1)), BIC(([0 if d is None else d for d in gpa_cdf1]), yed1, len(gpa_fit1)))
print('BIC-Q_S2')
print(BIC(gum_cdf2, yed2, len(gum_fit2)), BIC(gev_cdf2, yed2, len(gev_fit2)), BIC(([0 if d is None else d for d in gpa_cdf2]), yed2, len(gpa_fit2)))

Set 3

In [ ]:
#Marginal distributions - Parameter estimation
                                        #Method of fitting 'MM' Method of moments 'MLE' Maximum Likelihood Estimation

LMU1 = lmom.samlmu(AM3['Q_S1'])
LMU2 = lmom.samlmu(AM3['Q_S2'])
#Gumbel
gum_fit1 = lmom.pelgum(LMU1)
gum_fit2 = lmom.pelgum(LMU2)
# GEV
gev_fit1 = lmom.pelgev(LMU1)
gev_fit2 = lmom.pelgev(LMU2)
#General pareto
gpa_fit1 = lmom.pelgpa(LMU1)
gpa_fit2 = lmom.pelgpa(LMU2)
#Normal
nor_fit1 = lmom.pelnor(LMU1)
nor_fit2 = lmom.pelnor(LMU2)
#Empirical
xed1, yed1 = ecdf(AM3['Q_S1'])
xed2, yed2 = ecdf(AM3['Q_S2'])

#CDFs
gum_cdf1 = np.array([])
gum_cdf2 = np.array([])
gev_cdf1 = np.array([])
gev_cdf2 = np.array([])
gpa_cdf1 = np.array([])
gpa_cdf2 = np.array([])
nor_cdf1 = np.array([])
nor_cdf2 = np.array([])

# x1 = np.linspace(AM3['Q_S1'].min(), AM3['Q_S1'].max(), 100)
# x2 = np.linspace(AM3['Q_S2'].min(), AM3['Q_S2'].max(), 100)

for i in range(len(xed1)):
    #Gumbel
    gum_cdf1 = np.hstack((gum_cdf1, lmom.cdfgum(xed1[i], gum_fit1)))
    gum_cdf2 = np.hstack((gum_cdf2, lmom.cdfgum(xed2[i], gum_fit2)))
    #GEV
    gev_cdf1 = np.hstack((gev_cdf1, lmom.cdfgev(xed1[i], gev_fit1)))
    gev_cdf2 = np.hstack((gev_cdf2, lmom.cdfgev(xed2[i], gev_fit2)))
    #General pareto
    gpa_cdf1 = np.hstack((gpa_cdf1, lmom.cdfgpa(xed1[i], gpa_fit1)))
    gpa_cdf2 = np.hstack((gpa_cdf2, lmom.cdfgpa(xed2[i], gpa_fit2)))
    #Normal 
    nor_cdf1 = np.hstack((nor_cdf1, lmom.cdfnor(xed1[i], nor_fit1)))
    nor_cdf2 = np.hstack((nor_cdf2, lmom.cdfnor(xed2[i], nor_fit2)))


cdfs_ = gum_cdf1, gum_cdf2, gev_cdf1, gev_cdf2, gpa_cdf1, gpa_cdf2, nor_cdf1, nor_cdf2
cdfs_text = ['gum_cdf1', 'gum_cdf2', 'gev_cdf1', 'gev_cdf2', 'gpa_cdf1', 'gpa_cdf2', 'nor_cdf1', 'nor_cdf2']
RMSE_ = RMSE(gum_cdf1, yed1), RMSE(gum_cdf2, yed2), RMSE(gev_cdf1, yed1), RMSE(gev_cdf2, yed2), RMSE(([0 if d is None else d for d in gpa_cdf1]), yed1), RMSE(([0 if d is None else d for d in gpa_cdf2]), yed2), RMSE(nor_cdf1, yed1), RMSE(nor_cdf2, yed2)
AIC_ = AIC(gum_cdf1, yed1, len(gum_fit1)), AIC(gum_cdf2, yed2, len(gum_fit2)), AIC(gev_cdf1, yed1, len(gev_fit1)), AIC(gev_cdf2, yed2, len(gev_fit2)), AIC(([0 if d is None else d for d in gpa_cdf1]), yed1, len(gpa_fit1)), AIC(([0 if d is None else d for d in gpa_cdf2]), yed2, len(gpa_fit2)), AIC(nor_cdf1, yed1, len(nor_fit1)), AIC(nor_cdf2, yed2, len(nor_fit2))
BIC_ = BIC(gum_cdf1, yed1, len(gum_fit1)), BIC(gum_cdf2, yed2, len(gum_fit2)), BIC(gev_cdf1, yed1, len(gev_fit1)), BIC(gev_cdf2, yed2, len(gev_fit2)), BIC(([0 if d is None else d for d in gpa_cdf1]), yed1, len(gpa_fit1)), BIC(([0 if d is None else d for d in gpa_cdf2]), yed2, len(gpa_fit2)), BIC(nor_cdf1, yed1, len(nor_fit1)), BIC(nor_cdf2, yed2, len(nor_fit2))

Title = 'Gumbel (GUM)', 'Generalised Extreme Value (GEV)', 'Generalised Pareto (GPA)', 'Normal (NOR)'
plt.figure(figsize=(12, 12))
plt.suptitle('Set 3 | Cmax (C=S1+S2)', fontsize=22, y=1.02) #'Set 2 | S2max-S1conc' 'Set 3 | Cmax (C=S1+S2)'
# plt.title('Set 1 | S1max-S2conc', fontsize=22) 
#for i in dis_:
count1 = -1
count2 = 0
for j in range(4):
    count1 = count1 + 2
    plt.subplot(4,2, count1)
    plt.plot(xed1, yed1, 'm+', label='Empirical distribution 1')
    plt.plot(xed1, cdfs_[count1-1], 'b', label=f'{cdfs_text[count1-1]} \nRMSE={RMSE_[count1-1]:.4f} \nAIC={AIC_[count1-1]:.0f} \nBIC={BIC_[count1-1]:.0f}')
    plt.title(f'{Title[j]} | Rhine river', fontsize=18)
    plt.xlabel('Discharge (m3/s)', fontsize=14)
    plt.ylabel('Probability', fontsize=14) #Q m3/s
    plt.legend(fontsize=14, loc=4)
    plt.grid()
    count2 = count2 + 2
    plt.subplot(4,2, count2)
    plt.plot(xed2, yed2, 'm+', label='Empirical distribution 2')
    plt.plot(xed2, cdfs_[count2-1], 'b', label=f'{cdfs_text[count2-1]} \nRMSE={RMSE_[count2-1]:.4f} \nAIC={AIC_[count2-1]:.0f} \nBIC={BIC_[count2-1]:.0f}')
    plt.title(f'{Title[j]} | Main river', fontsize=18)
    plt.xlabel('Discharge (m3/s)', fontsize=14)
    plt.ylabel('Probability', fontsize=14) #Q m3/s
    plt.legend(fontsize=14, loc=4)
    plt.grid()
plt.tight_layout()

In [ ]:
#CDFs
gum_cdf1 = np.array([])
gum_cdf2 = np.array([])
gev_cdf1 = np.array([])
gev_cdf2 = np.array([])
gpa_cdf1 = np.array([])
gpa_cdf2 = np.array([])
nor_cdf1 = np.array([])
nor_cdf2 = np.array([])

x1 = np.linspace(AM1['Q_S1'].min(), AM1['Q_S1'].max()+1000, 100)
x2 = np.linspace(AM1['Q_S2'].min(), AM1['Q_S2'].max()+1000, 100)

for i in range(100):
    #Gumbel
    gum_cdf1 = np.hstack((gum_cdf1, lmom.cdfgum(x1[i], gum_fit1)))
    gum_cdf2 = np.hstack((gum_cdf2, lmom.cdfgum(x2[i], gum_fit2)))
    #GEV
    gev_cdf1 = np.hstack((gev_cdf1, lmom.cdfgev(x1[i], gev_fit1)))
    gev_cdf2 = np.hstack((gev_cdf2, lmom.cdfgev(x2[i], gev_fit2)))
    #General pareto
    gpa_cdf1 = np.hstack((gpa_cdf1, lmom.cdfgpa(x1[i], gpa_fit1)))
    gpa_cdf2 = np.hstack((gpa_cdf2, lmom.cdfgpa(x2[i], gpa_fit2)))
    #Normal
    nor_cdf1 = np.hstack((nor_cdf1, lmom.cdfnor(x1[i], nor_fit1)))
    nor_cdf2 = np.hstack((nor_cdf2, lmom.cdfnor(x2[i], nor_fit2)))

gpa_cdf1_ = np.zeros(len(gpa_cdf1))
gpa_cdf2_ = np.zeros(len(gpa_cdf2))

for i in range(len(gpa_cdf1_)):
    if gpa_cdf1[i] == None:
        gpa_cdf1_[i] = 0.00001
    else:
        gpa_cdf1_[i] = gpa_cdf1[i]
    if gpa_cdf2[i] == None:
        gpa_cdf2_[i] = 0.00001
    else:
        gpa_cdf2_[i] = gpa_cdf2[i]
# gpa_cdf1_ = ([0 if d is None else d for d in gpa_cdf1])
# gpa_cdf2_ = ([0 if d is None else d for d in gpa_cdf2])

#print(gpa_cdf2)
cdfs_ = gum_cdf1, gum_cdf2, gev_cdf1, gev_cdf2, gpa_cdf1_, gpa_cdf2_, nor_cdf1, nor_cdf2
cdfs_text = ['gum_cdf1', 'gum_cdf2', 'gev_cdf1', 'gev_cdf2', 'gpa_cdf1', 'gpa_cdf2', 'nor_cdf1', 'nor_cdf2']


Title = 'Gumbel (GUM)', 'Generalised Extreme Value (GEV)', 'Generalised Pareto (GPA)', 'Normal (NOR)'
plt.figure(figsize=(12, 12))
plt.suptitle('Set 3 | Cmax (C=S1+S2)', fontsize=22, y=1.02) #'Set 2 | S2max-S1conc' 'Set 3 | Cmax (C=S1+S2)'
#for i in dis_:
count1 = -1
count2 = 0
for j in range(4):
    count1 = count1 + 2
    plt.subplot(4,2, count1)
    plt.plot(1/(1-cdfs_[count1-1]), x1, 'b', label=f'{cdfs_text[count1-1]} \nRMSE={RMSE_[count1-1]:.4f} \nAIC={AIC_[count1-1]:.0f} \nBIC={BIC_[count1-1]:.0f}')
    plt.plot(1/(1-yed1), xed1, 'm+', label='Empirical distribution 1')
    plt.title(f'{Title[j]} | Rhine river', fontsize=18)
    plt.xlabel('Return period (years)')
    plt.ylabel('Discharge (m3/s)')
    plt.xscale('log')
    plt.xlim(right=10**6)
    plt.legend()
    plt.grid()
    count2 = count2 + 2
    plt.subplot(4,2, count2)
    plt.plot(1/(1-cdfs_[count2-1][4:]), x2[4:], 'b', label=f'{cdfs_text[count2-1]} \nRMSE={RMSE_[count2-1]:.4f} \nAIC={AIC_[count2-1]:.0f} \nBIC={BIC_[count2-1]:.0f}')
    plt.plot(1/(1-yed2), xed2, 'm+', label='Empirical distribution 2')
    plt.title(f'{Title[j]} | Main river', fontsize=18)
    plt.xlabel('Return period (years)')
    plt.ylabel('Discharge (m3/s)')
    plt.xscale('log')
    plt.xlim(right=10**6)
    plt.legend()
    plt.grid()
plt.tight_layout()

In [ ]:
print('RMSE-Q_S1')
print(RMSE(gum_cdf1, yed1), RMSE(gev_cdf1, yed1), RMSE(([0 if d is None else d for d in gpa_cdf1]), yed1))
print('RMSE-Q_S2')
print(RMSE(gum_cdf2, yed2), RMSE(gev_cdf2, yed2), RMSE(([0 if d is None else d for d in gpa_cdf2]), yed2))
print('AIC-Q_S1')
print(AIC(gum_cdf1, yed1, len(gum_fit1)), AIC(gev_cdf1, yed1, len(gev_fit1)), AIC(([0 if d is None else d for d in gpa_cdf1]), yed1, len(gpa_fit1)))
print('AIC-Q_S2')
print(AIC(gum_cdf2, yed2, len(gum_fit2)), AIC(gev_cdf2, yed2, len(gev_fit2)), AIC(([0 if d is None else d for d in gpa_cdf2]), yed2, len(gpa_fit2)))
print('BIC-Q_S1')
print(BIC(gum_cdf1, yed1, len(gum_fit1)), BIC(gev_cdf1, yed1, len(gev_fit1)), BIC(([0 if d is None else d for d in gpa_cdf1]), yed1, len(gpa_fit1)))
print('BIC-Q_S2')
print(BIC(gum_cdf2, yed2, len(gum_fit2)), BIC(gev_cdf2, yed2, len(gev_fit2)), BIC(([0 if d is None else d for d in gpa_cdf2]), yed2, len(gpa_fit2)))